# General

In [1]:
import numpy as np
import mlflow
import json
from mlflow.tracking import MlflowClient

# Initialisiere den Client
client = MlflowClient('/pfs/work7/workspace/scratch/ma_aansari-team_project_fss2024_de/mlflow')

In [56]:
common_corruptions = [
    'gaussian_noise',
  'shot_noise',
  'impulse_noise',
  'defocus_blur',
  'glass_blur',
  'motion_blur',
  'zoom_blur',
  'snow',
  'frost',
  'fog',
  'brightness',
  'contrast',
  'elastic_transform',
  'pixelate',
  'jpeg_compression'
]

severities = range(1, 6)

attacks = [
  'fgsm',
  'pgd',
  'apgd',
  'bim',
  'cospgd'
]

iterations = range(1, 20)

metrics = {
    'test_EPE_0':['cfnet', 'gwcnet-g'], 
    'test_Thres1_0':['cfnet', 'gwcnet-g'], 
    'test_Thres2_0':['cfnet', 'gwcnet-g'], 
    'test_Thres3_0':['cfnet', 'gwcnet-g'], 
    'test_D1_0':['cfnet', 'gwcnet-g'],
    'eval/3px_error':['sttr', 'sttr-light'],
    'eval/epe':['sttr', 'sttr-light'],
    'eval/iou':['sttr', 'sttr-light'],
    'eval/l1':['sttr', 'sttr-light'],
    'eval/l1_raw':['sttr', 'sttr-light'],
    'eval/occ_be':['sttr', 'sttr-light'],
    'eval/rr':['sttr', 'sttr-light'],
    }

datasets = ['sceneflow', 'kitti2015']
architectures = ['cfnet', 'gwcnet-g', 'sttr', 'sttr-light']

# Assuming you have a way to fetch the mlflow data, for example, using mlflow.search_runs()
# Here, we will create a mock structure as an example

In [57]:
def get_experiment_id(architecture, dataset, experiment_type='Common_Corruptions'):
    experiment_name = f'{experiment_type}_{architecture}_{dataset}'
    experiment_id = None

    experiment = client.get_experiment_by_name(experiment_name)
    if experiment:
        print(f"Experiment mit dem Namen {experiment_name} gefunden.")
        return experiment.experiment_id
    else:
        print(f"Experiment mit dem Namen {experiment_name} nicht gefunden.")
        return None

In [58]:
def get_run_id(architecture:str, dataset:str, run_type:str, experiment_id:int, corruption_type:str=None, severity:int=None, attack_name:str=None, metric_name:str=None):

    if run_type == 'commoncorruption':
        assert corruption_type in common_corruptions
        assert severity in severities
        
        run_name = f"{architecture}_{run_type}_{dataset}_{corruption_type}_{severity}"
    elif run_type == 'attack':
        assert attack_name in attacks
        run_name = f"{architecture}_{run_type}_{dataset}_{attack_name}"

    runs = client.search_runs(experiment_id, filter_string=f"tags.mlflow.runName = '{run_name}'", max_results=1)

    # Stelle sicher, dass der Run gefunden wurde
    if runs:
        print(f"Run mit dem Namen {run_name} gefunden.")
        return runs[0].info.run_id
    else:
        print(f"Run mit dem Namen {run_name} nicht gefunden.")
        return None

In [59]:
# Method to save time (only once loading experiment_id and run_id which is time consuming)
def get_mlflow_metadata():
    
    metadata = []
    # global metadata

    for dataset in datasets:
        for architecture in architectures:
            if dataset == 'kitti2015' and architecture in ['sttr-light', 'cfnet']:
                continue
            experiment_id = get_experiment_id(architecture, dataset)
            
            for common_corruption in common_corruptions:
                for severity in severities:
            
                    run_id = get_run_id(architecture, dataset, experiment_id=experiment_id, run_type="common_corruption", common_corruption=common_corruption, severity=severity)
                    print(f"Run ID: {run_id}, common_corruption: {common_corruption}, severity: {severity}")
                    metadata.append({
                        'experiment_id': experiment_id,
                        'run_id': run_id,
                        'dataset': dataset,
                        'architecture': architecture,
                        'common_corruption': common_corruption,
                        'severity': severity
                    })


            # also add no_corruption and severity 0
            common_corruption = 'no_corruption'
            severity = 0

            run_id = get_run_id(architecture, dataset, experiment_id=experiment_id, run_type="common_corruption", common_corruption=common_corruption, severity=severity)
            print(f"Run ID: {run_id}, common_corruption: {common_corruption}, severity: {severity}")
            metadata.append({
                'experiment_id': experiment_id,
                'run_id': run_id,
                'dataset': dataset,
                'architecture': architecture,
                'common_corruption': common_corruption,
                'severity': severity
            })

    # Store mlflow_data as JSON
    with open('metadata.json', 'w') as f:
        json.dump(metadata, f)
    
    return metadata

In [60]:
# metadata = get_mlflow_metadata()
metadata = json.load(open('metadata.json'))

In [61]:
def get_run_id_from_metadata(metadata, architecture, dataset, common_corruption, severity):
    
    for data in metadata:
        if data['dataset'] == dataset and data['architecture'] == architecture and data['common_corruption'] == common_corruption and data['severity'] == severity:
            return data['run_id']
            
    raise ValueError(f"Run not found for dataset={dataset}, architecture={architecture}, common_corruption={common_corruption}, severity={severity}")

In [82]:
def get_empty_mlflow_data_dictionary(runtype:str):

    assert runtype in ['commoncorruption', 'attack']

    mlflow_data = {
        'architecture': [], 
        'backbone': [], 
        'type_backbone': [],
        'dataset': []
    }

    for metric in metrics.keys():

        if runtype == 'commoncorruption':
            for common_corruption in common_corruptions:
                for severity in severities:
                    mlflow_data[f'{metric}-{common_corruption}-sev{severity}'] = []

            # also add no_corruption and severity 0
            severity = 0
            common_corruption = 'no_corruption'
            mlflow_data[f'{metric}-{common_corruption}-sev{severity}'] = []
        else:
            for attack in attacks:
                for iteration in iterations:
                    
                    metric_splitted = metric.split('_')
                    metric_iteration = metric_splitted[0] + "_" + str(iteration) + "_" + '_'.join(metric_splitted[1:])
                    mlflow_data[f'{metric_iteration}-{attack}'] = []
                
    return mlflow_data

# Export commoncorruption data

In [ ]:
mlflow_data = get_empty_mlflow_data_dictionary('commoncorruption')

{'architecture': [], 'backbone': [], 'type_backbone': [], 'dataset': [], 'test_EPE_0-gaussian_noise-sev1': [], 'test_EPE_0-gaussian_noise-sev2': [], 'test_EPE_0-gaussian_noise-sev3': [], 'test_EPE_0-gaussian_noise-sev4': [], 'test_EPE_0-gaussian_noise-sev5': [], 'test_EPE_0-shot_noise-sev1': [], 'test_EPE_0-shot_noise-sev2': [], 'test_EPE_0-shot_noise-sev3': [], 'test_EPE_0-shot_noise-sev4': [], 'test_EPE_0-shot_noise-sev5': [], 'test_EPE_0-impulse_noise-sev1': [], 'test_EPE_0-impulse_noise-sev2': [], 'test_EPE_0-impulse_noise-sev3': [], 'test_EPE_0-impulse_noise-sev4': [], 'test_EPE_0-impulse_noise-sev5': [], 'test_EPE_0-defocus_blur-sev1': [], 'test_EPE_0-defocus_blur-sev2': [], 'test_EPE_0-defocus_blur-sev3': [], 'test_EPE_0-defocus_blur-sev4': [], 'test_EPE_0-defocus_blur-sev5': [], 'test_EPE_0-glass_blur-sev1': [], 'test_EPE_0-glass_blur-sev2': [], 'test_EPE_0-glass_blur-sev3': [], 'test_EPE_0-glass_blur-sev4': [], 'test_EPE_0-glass_blur-sev5': [], 'test_EPE_0-motion_blur-sev1': [

In [54]:
def add_corruption_commoncorruption_severity(architecture, dataset, common_corruption, severity):
    # filter metadata to get the run_id
    run_id = get_run_id_from_metadata(metadata, architecture, dataset, common_corruption, severity)

    for metric, metric_architectures in metrics.items():
        
        # Schritt : Die spezifische Metrik abrufen
        metric_history = client.get_metric_history(run_id, metric)
        
        # Die Werte der Metrik abrufen
        values = [m.value for m in metric_history]
        metric_name = f"{metric}-{common_corruption}-sev{severity}"
        value = None
        
        if architecture not in metric_architectures:
            value = "-"
        elif len(values) == 0:
            raise ValueError(f"No values found for {metric_name}, dataset: {dataset}, architecture: {architecture}, corruption: {common_corruption}, severity: {severity}, run_id: {run_id}")
        elif len(values) == 1:
            value = values[0]
        else:
            value = np.mean(values)

        print(f"\t\t\t\t{metric_name}: {value}")
    
        
        mlflow_data[metric_name].append(value)

In [55]:
def add_corruption(architecture, dataset):
    
    mlflow_data['architecture'].append(architecture)
    mlflow_data['backbone'].append("-")
    mlflow_data['type_backbone'].append("-")
    mlflow_data['dataset'].append(dataset)

    print(f"Architecture: {architecture}, Dataset: {dataset}")
    # check for all keys in the metrics if the length of the values are equal
    
    
    for common_corruption in common_corruptions:
        print("\t\t", common_corruption)
        
        for severity in severities:
            print("\t\t\t", severity)

            add_corruption_commoncorruption_severity(architecture, dataset, common_corruption, severity)

    # also add no_corruption and severity 0
    common_corruption = 'no_corruption'
    severity = 0
    print("\t\t", common_corruption)
    print("\t\t\t", severity)
    add_corruption_commoncorruption_severity(architecture, dataset, common_corruption, severity)
    

    for key, value in mlflow_data.items():
        if(len(value) != len(mlflow_data['architecture'])):
            raise ValueError(f"Length of values for {key} is not equal to the length of the architecture., length of values: {len(value)}, length of arc: {len(mlflow_data['architecture'])}")


In [56]:
for dataset in datasets:
    print(dataset)
    # print(len(mlflow_data['test_EPE_0']))
    for architecture in architectures:
        if dataset == 'kitti2015' and architecture in ['sttr-light', 'cfnet']:
                continue
        print("\t", architecture)
        add_corruption(
            architecture=architecture, 
            dataset=dataset)

sceneflow
	 cfnet
Architecture: cfnet, Dataset: sceneflow
		 gaussian_noise
			 1
				test_EPE_0-gaussian_noise-sev1: 29.128144017257785
				test_Thres1_0-gaussian_noise-sev1: 0.6289255706887497
				test_Thres2_0-gaussian_noise-sev1: 0.5192185411514495
				test_Thres3_0-gaussian_noise-sev1: 0.46389480859115906
				test_D1_0-gaussian_noise-sev1: 0.4506077169438353
				eval/3px_error-gaussian_noise-sev1: -
				eval/epe-gaussian_noise-sev1: -
				eval/iou-gaussian_noise-sev1: -
				eval/l1-gaussian_noise-sev1: -
				eval/l1_raw-gaussian_noise-sev1: -
				eval/occ_be-gaussian_noise-sev1: -
				eval/rr-gaussian_noise-sev1: -
			 2
				test_EPE_0-gaussian_noise-sev2: 102.75801340426133
				test_Thres1_0-gaussian_noise-sev2: 0.8956249203916545
				test_Thres2_0-gaussian_noise-sev2: 0.8505774473980308
				test_Thres3_0-gaussian_noise-sev2: 0.8238480773343102
				test_D1_0-gaussian_noise-sev2: 0.8167968914219662
				eval/3px_error-gaussian_noise-sev2: -
				eval/epe-gaussian_noise-sev2: -
				e

In [60]:
mlflow_data

{'architecture': ['cfnet',
  'gwcnet-g',
  'sttr',
  'sttr-light',
  'gwcnet-g',
  'sttr'],
 'backbone': ['-', '-', '-', '-', '-', '-'],
 'type_backbone': ['-', '-', '-', '-', '-', '-'],
 'dataset': ['sceneflow',
  'sceneflow',
  'sceneflow',
  'sceneflow',
  'kitti2015',
  'kitti2015'],
 'test_EPE_0-gaussian_noise-sev1': [29.128144017257785,
  8.934847301858513,
  '-',
  '-',
  4.179471969604492,
  '-'],
 'test_EPE_0-gaussian_noise-sev2': [102.75801340426133,
  17.085862001838073,
  '-',
  '-',
  3.9743337631225586,
  '-'],
 'test_EPE_0-gaussian_noise-sev3': [176.7278970155345,
  41.555038244827934,
  '-',
  '-',
  3.499112844467163,
  '-'],
 'test_EPE_0-gaussian_noise-sev4': [188.0737592020624,
  85.62713641989421,
  '-',
  '-',
  5.161510944366455,
  '-'],
 'test_EPE_0-gaussian_noise-sev5': [188.76334761351143,
  115.44809951956812,
  '-',
  '-',
  7.098750114440918,
  '-'],
 'test_EPE_0-shot_noise-sev1': [21.224237462912328,
  8.893984444572396,
  '-',
  '-',
  4.271886348724365,
 

In [61]:
for key, value in mlflow_data.items():
    print(key, len(value))

architecture 6
backbone 6
type_backbone 6
dataset 6
test_EPE_0-gaussian_noise-sev1 6
test_EPE_0-gaussian_noise-sev2 6
test_EPE_0-gaussian_noise-sev3 6
test_EPE_0-gaussian_noise-sev4 6
test_EPE_0-gaussian_noise-sev5 6
test_EPE_0-shot_noise-sev1 6
test_EPE_0-shot_noise-sev2 6
test_EPE_0-shot_noise-sev3 6
test_EPE_0-shot_noise-sev4 6
test_EPE_0-shot_noise-sev5 6
test_EPE_0-impulse_noise-sev1 6
test_EPE_0-impulse_noise-sev2 6
test_EPE_0-impulse_noise-sev3 6
test_EPE_0-impulse_noise-sev4 6
test_EPE_0-impulse_noise-sev5 6
test_EPE_0-defocus_blur-sev1 6
test_EPE_0-defocus_blur-sev2 6
test_EPE_0-defocus_blur-sev3 6
test_EPE_0-defocus_blur-sev4 6
test_EPE_0-defocus_blur-sev5 6
test_EPE_0-glass_blur-sev1 6
test_EPE_0-glass_blur-sev2 6
test_EPE_0-glass_blur-sev3 6
test_EPE_0-glass_blur-sev4 6
test_EPE_0-glass_blur-sev5 6
test_EPE_0-motion_blur-sev1 6
test_EPE_0-motion_blur-sev2 6
test_EPE_0-motion_blur-sev3 6
test_EPE_0-motion_blur-sev4 6
test_EPE_0-motion_blur-sev5 6
test_EPE_0-zoom_blur-sev1 6


In [62]:
# Store mlflow_data as JSON
with open('mlflow_data.json', 'w') as f:
    json.dump(mlflow_data, f)

# Attacks

In [87]:
mlflow_data = get_empty_mlflow_data_dictionary(runtype='attack')

runs = []

for architecture in architectures:
    for dataset in datasets:
        experiment_id = get_experiment_id(architecture, dataset, 'Attacks')
        if experiment_id:
            #  print(experiment_id)

            for attack in attacks:
                run_id = get_run_id(architecture, dataset, experiment_id=experiment_id, run_type="attack", attack_name=attack)
                # print(f"Run ID: {run_id}, attack: {attack}")

                if run_id:
                    print(f"Run ID: {run_id}, attack: {attack}, dataset: {dataset}, architecture: {architecture}")
                    runs.append({
                        'run_id': run_id,
                        'attack': attack,
                        'dataset': dataset,
                        'architecture': architecture
                    }) 
                continue
                print

                for metric, metric_architectures in metrics.items():

                    for iteration in iterations:

                        metric_splitted = metric.split('_')
                        metric_iteration = metric_splitted[0] + "_" + str(iteration) + "_" + '_'.join(metric_splitted[1:])
                        
                        metric_history = client.get_metric_history(run_id, metric_iteration)
                        values = [m.value for m in metric_history]
                        value = None

                        if architecture not in metric_architectures:
                            value = "-"
                        elif len(values) == 0:
                            raise ValueError(f"No values found for {metric_iteration}, dataset: {dataset}, architecture: {architecture}, attack: {attack}, run_id: {run_id}")
                        elif len(values) == 1:
                            value = values[0]
                        else:
                            value = np.mean(values)

                        print(f"\t\t\t\t{metric_iteration}: {value}")

                        mlflow_data[f'{metric_iteration}-{attack}'].append(value)

Experiment mit dem Namen Attacks_cfnet_sceneflow nicht gefunden.
Experiment mit dem Namen Attacks_cfnet_kitti2015 nicht gefunden.
Experiment mit dem Namen Attacks_gwcnet-g_sceneflow nicht gefunden.
Experiment mit dem Namen Attacks_gwcnet-g_kitti2015 gefunden.
Run mit dem Namen gwcnet-g_attack_kitti2015_fgsm nicht gefunden.
Run mit dem Namen gwcnet-g_attack_kitti2015_pgd gefunden.
Run ID: 78e64fe8c9014075b95ba823434c8da9, attack: pgd, dataset: kitti2015, architecture: gwcnet-g
Run mit dem Namen gwcnet-g_attack_kitti2015_apgd gefunden.
Run ID: a065a02ab72d461cb95c4de86684cabb, attack: apgd, dataset: kitti2015, architecture: gwcnet-g
Run mit dem Namen gwcnet-g_attack_kitti2015_bim gefunden.
Run ID: 4e1d73e103114ef6b1aae7a2357fc1bb, attack: bim, dataset: kitti2015, architecture: gwcnet-g
Run mit dem Namen gwcnet-g_attack_kitti2015_cospgd nicht gefunden.
Experiment mit dem Namen Attacks_sttr_sceneflow nicht gefunden.
Experiment mit dem Namen Attacks_sttr_kitti2015 nicht gefunden.
Experiment

In [105]:
run_id = get_run_id(architecture='gwcnet-g', dataset='kitti2015', run_type='attack', experiment_id='569013368659275458', attack_name='fgsm')

runs.append({
    'run_id': run_id,
    'attack': 'fgsm',
    'dataset': 'kitti2015',
    'architecture': 'gwcnet-g'
})

Run mit dem Namen gwcnet-g_attack_kitti2015_fgsm gefunden.


In [92]:
runs

[{'run_id': '78e64fe8c9014075b95ba823434c8da9',
  'attack': 'pgd',
  'dataset': 'kitti2015',
  'architecture': 'gwcnet-g'},
 {'run_id': 'a065a02ab72d461cb95c4de86684cabb',
  'attack': 'apgd',
  'dataset': 'kitti2015',
  'architecture': 'gwcnet-g'},
 {'run_id': '4e1d73e103114ef6b1aae7a2357fc1bb',
  'attack': 'bim',
  'dataset': 'kitti2015',
  'architecture': 'gwcnet-g'},
 {'run_id': '1ca87d56129147aeb9cb65b03d3a73c8',
  'attack': 'cospgd',
  'dataset': 'kitti2015',
  'architecture': 'gwcnet-g'},
 {'run_id': 'd95ba6dd063843eea0efb3f7020f04fb',
  'attack': 'cospgd',
  'dataset': 'kitti2015',
  'architecture': 'gwcnet-g'}]

In [ ]:
for run in runs:
    for metric, metric_architectures in metrics.items():
        
        dataset = run['dataset']
        architecture = run['architecture']
        attack = run['attack']
        run_id = run['run_id']

        print(architecture, attack, dataset)
        

        if architecture not in metric_architectures:
            continue

        iteration = 19
        
        if attack == 'fgsm':
            iteration = 0
        

        metric_splitted = metric.split('_')
        metric_iteration = metric_splitted[0] + "_" + str(iteration) + "_" + '_'.join(metric_splitted[1:])
        
        metric_history = client.get_metric_history(run_id, metric_iteration)
        values = [m.value for m in metric_history]
        value = None

        if len(values) == 0:
            # value = "-"
            raise ValueError(f"No values found for {metric_iteration}, dataset: {dataset}, architecture: {architecture}, attack: {attack}, run_id: {run_id}")
        elif len(values) == 1:
            value = values[0]
        else:
            value = np.mean(values)

        print(f"\t{metric_iteration}: {value}")
        data[]

        # mlflow_data[f'{metric_iteration}-{attack}'].append(value)

gwcnet-g pgd kitti2015
	test_19_EPE_0: 201.14032062731292
gwcnet-g pgd kitti2015
	test_19_Thres1_0: 0.9997661835268924
gwcnet-g pgd kitti2015
	test_19_Thres2_0: 0.9985971607659992
gwcnet-g pgd kitti2015
	test_19_Thres3_0: 0.9973045179718419
gwcnet-g pgd kitti2015
	test_19_D1_0: 0.9973045179718419
gwcnet-g pgd kitti2015
gwcnet-g pgd kitti2015
gwcnet-g pgd kitti2015
gwcnet-g pgd kitti2015
gwcnet-g pgd kitti2015
gwcnet-g pgd kitti2015
gwcnet-g pgd kitti2015
gwcnet-g apgd kitti2015
	test_19_EPE_0: 189.7278410259046
gwcnet-g apgd kitti2015
	test_19_Thres1_0: 0.9999943532441792
gwcnet-g apgd kitti2015
	test_19_Thres2_0: 0.9999877026206568
gwcnet-g apgd kitti2015
	test_19_Thres3_0: 0.9999733473125257
gwcnet-g apgd kitti2015
	test_19_D1_0: 0.9999733473125257
gwcnet-g apgd kitti2015
gwcnet-g apgd kitti2015
gwcnet-g apgd kitti2015
gwcnet-g apgd kitti2015
gwcnet-g apgd kitti2015
gwcnet-g apgd kitti2015
gwcnet-g apgd kitti2015
gwcnet-g bim kitti2015
	test_19_EPE_0: 105.93373047678094
gwcnet-g bim 

In [104]:
runs

[{'run_id': '78e64fe8c9014075b95ba823434c8da9',
  'attack': 'pgd',
  'dataset': 'kitti2015',
  'architecture': 'gwcnet-g'},
 {'run_id': 'a065a02ab72d461cb95c4de86684cabb',
  'attack': 'apgd',
  'dataset': 'kitti2015',
  'architecture': 'gwcnet-g'},
 {'run_id': '4e1d73e103114ef6b1aae7a2357fc1bb',
  'attack': 'bim',
  'dataset': 'kitti2015',
  'architecture': 'gwcnet-g'},
 {'run_id': '1ca87d56129147aeb9cb65b03d3a73c8',
  'attack': 'cospgd',
  'dataset': 'kitti2015',
  'architecture': 'gwcnet-g'}]